<a href="https://colab.research.google.com/github/p-pashootan/Bioinformatics-course/blob/main/Array_express_Metastatic_melanoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gene expression data Analysis: Comparing Metastatic, Primary and Control Groups

Accession: E-GEOD-22301 (Array Express)


# Step 1: Library Installation and Loading

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

BiocManager::install(c("edgeR", "limma", "pheatmap"), force = TRUE)
install.packages("ggplot2")
install.packages("ggrepel")


library(edgeR)
library(limma)
library(pheatmap)
library(ggplot2)
library(ggrepel)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.25), R 4.5.0 (2025-04-11)

Installing package(s) 'BiocVersion', 'edgeR', 'limma', 'pheatmap'

also installing the dependencies ‘locfit’, ‘statmod’


Warning message in install.packages(...):
“installation of package ‘edgeR’ had non-zero exit status”
Old packages: 'cli', 'diffobj'

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Step 2: Read and merge the sample tables
Read and merge all sample .txt files.

In [ ]:
setwd("C:\\Users\\PARIYA\\Desktop\\New folder")


# Get a list of all .txt files
file_list <- list.files(pattern = "\\.txt$")

# Sort the file list for consistency
file_list <- sort(file_list)

# Initialize an empty list to store data frames
expression_list <- list()

# Loop through each file and read data
for (file in file_list) {
  # Read the tab-delimited file
  data <- read.table(file, header = TRUE, sep = "\t", stringsAsFactors = FALSE)

  # Rename the VALUE column to the sample name (based on filename)
  sample_name <- tools::file_path_sans_ext(basename(file))
  colnames(data) <- c("GeneID", sample_name)

  # Add to the list
  expression_list[[sample_name]] <- data
}

# Merge all data frames by 'GeneID'
merged_data <- Reduce(function(x, y) merge(x, y, by = "GeneID"), expression_list)
write.table(merged_data, file = "merged_expression_matrix.txt", sep = "\t", quote = FALSE, row.names = FALSE)

# Step 3: Read the merged expression matrix and define groups

In [ ]:
expr_data <- read.table("merged_expression_matrix.txt", header = TRUE, sep = "\t", row.names = 1)

# Replace with your actual column/sample names
sample_names <- colnames(expr_data)

# Manually define which samples belong to which group
control_samples <- c("GSM555116_sample_table", "GSM555117_sample_table", "GSM555118_sample_table", "GSM555135_sample_table")
metastatic_samples <- c("GSM555120_sample_table", "GSM555121_sample_table", "GSM555122_sample_table", "GSM555123_sample_table",
                        "GSM555124_sample_table", "GSM555125_sample_table", "GSM555126_sample_table", "GSM555127_sample_table",
                        "GSM555129_sample_table", "GSM555130_sample_table", "GSM555131_sample_table", "GSM555132_sample_table",
                        "GSM555128_sample_table", "GSM555137_sample_table")
primary_samples <- c("GSM555133_sample_table", "GSM555136_sample_table", "GSM555119_sample_table", "GSM555134_sample_table")


In [ ]:
# Create a named vector for group numbers
group_num <- rep(NA, length(sample_names))
names(group_num) <- sample_names

# Assign numbers to each group
group_num[control_samples] <- 1
group_num[primary_samples] <- 2
group_num[metastatic_samples] <- 3

# Create the final matrix
group_matrix <- data.frame(
  Index = 1:22,
  SampleID = sample_names,
  Group = group_num
)
print(group_matrix, row.names = FALSE)

# Step 4: Create DGEList and Normalize Data

In [ ]:
counts <- as.matrix(expr_data)
dge <- DGEList(counts = counts)

# Convert group_num to factor with meaningful labels
group_factor <- factor(group_num, levels = c(1, 2, 3),
                       labels = c("Control", "Primary", "Metastatic"))

# Create design matrix
design <- model.matrix(~ 0 + group_factor)
colnames(design) <- levels(group_factor)  # Rename columns for clarity
print(design)


In [ ]:
# Perform Voom Transformation
v <- voom(dge, design, plot=TRUE)

# Fit the Linear Model
fit <- lmFit(v, design)

# Step 5: Define Contrasts for Comparisons

In [ ]:
# Convert numeric group to factor
group_factor <- factor(group_num, levels = c(1, 2, 3), labels = c("Control", "Primary", "Metastatic"))


# Define contrasts
contrast.matrix <- makeContrasts(
  Met_vs_Control = Metastatic - Control,
  Met_vs_Primary = Metastatic - Primary,
  levels = design
)

print(contrast.matrix)
# Fit contrasts and run eBayes
fit2 <- contrasts.fit(fit, contrast.matrix)
fit2 <- eBayes(fit2)

# View top differentially expressed genes for each contrast
topTable(fit2, coef = "Met_vs_Control")
topTable(fit2, coef = "Met_vs_Primary")

# Step 6: Get all DEGs

In [ ]:
# Get all DEGs for Metastatic vs Control
deg_met_vs_ctrl <- topTable(fit2, coef = "Met_vs_Control", number = Inf)

# Get all DEGs for Metastatic vs Primary
deg_met_vs_prim <- topTable(fit2, coef = "Met_vs_Primary", number = Inf)

# Write to text files
write.table(deg_met_vs_ctrl, file = "DEGs_Met_vs_Control.txt", sep = "\t", quote = FALSE, row.names = TRUE)
write.table(deg_met_vs_prim, file = "DEGs_Met_vs_Primary.txt", sep = "\t", quote = FALSE, row.names = TRUE)


# Step 7: Create Visualizations

In [ ]:
volcano_plot <- function(data, title, logFC_cutoff = 1, pval_cutoff = 0.05) {
  data$Significant <- "Not Sig."
  data$Significant[data$logFC >= logFC_cutoff & data$adj.P.Val < pval_cutoff] <- "Up"
  data$Significant[data$logFC <= -logFC_cutoff & data$adj.P.Val < pval_cutoff] <- "Down"

  ggplot(data, aes(x = logFC, y = -log10(adj.P.Val), color = Significant)) +
    geom_point(alpha = 0.8) +
    scale_color_manual(values = c("blue", "grey", "red")) +
    theme_minimal() +
    labs(title = title, x = "log2 Fold Change", y = "-log10 Adjusted P-Value") +
    geom_vline(xintercept = c(-logFC_cutoff, logFC_cutoff), linetype = "dashed", color = "black") +
    geom_hline(yintercept = -log10(pval_cutoff), linetype = "dashed", color = "black")
}
volcano_plot(deg_met_vs_ctrl, "Volcano Plot: Metastatic vs Control")
volcano_plot(deg_met_vs_prim, "Volcano Plot: Metastatic vs Primary")

In [ ]:
# Select top 10 DEGs by adjusted p-value
top_genes_ctrl <- rownames(deg_met_vs_ctrl)[1:10]
top_genes_prim <- rownames(deg_met_vs_prim)[1:10]

In [ ]:
# Subset expression data for heatmap
expr_matrix_ctrl <- expr_data[top_genes_ctrl, c(control_samples, metastatic_samples)]
expr_matrix_prim <- expr_data[top_genes_prim, c(primary_samples, metastatic_samples)]

# Normalize
expr_matrix_ctrl_scaled <- t(scale(t(as.matrix(expr_matrix_ctrl))))
expr_matrix_prim_scaled <- t(scale(t(as.matrix(expr_matrix_prim))))

# Create annotation
annotation_ctrl <- data.frame(Group = factor(c(rep("Control", length(control_samples)), rep("Metastatic", length(metastatic_samples)))))
rownames(annotation_ctrl) <- c(control_samples, metastatic_samples)

annotation_prim <- data.frame(Group = factor(c(rep("Primary", length(primary_samples)), rep("Metastatic", length(metastatic_samples)))))
rownames(annotation_prim) <- c(primary_samples, metastatic_samples)

# Draw heatmaps
pheatmap(expr_matrix_ctrl_scaled, annotation_col = annotation_ctrl, main = "Top 10 DEGs: Metastatic vs Control")
pheatmap(expr_matrix_prim_scaled, annotation_col = annotation_prim, main = "Top 10 DEGs: Metastatic vs Primary")

# Step 8: Save Plots as PNGs

In [ ]:


png("Volcano_Metastatic_vs_Control.png", width = 1200, height = 1000, res = 150)
volcano_plot(deg_met_vs_ctrl, "Volcano Plot: Metastatic vs Control")
dev.off()
png("Volcano_Metastatic_vs_Primary.png", width = 1200, height = 1000, res = 150)
volcano_plot(deg_met_vs_prim, "Volcano Plot: Metastatic vs Primary")
dev.off()

png("Heatmap_Top10_Met_vs_Control.png", width = 1200, height = 1000, res = 150)
pheatmap(expr_matrix_ctrl_scaled, annotation_col = annotation_ctrl, main = "Top 10 DEGs: Metastatic vs Control")
dev.off()
png("Heatmap_Top10_Met_vs_Primary.png", width = 1200, height = 1000, res = 150)
pheatmap(expr_matrix_prim_scaled, annotation_col = annotation_ctrl, main = "Top 10 DEGs: Metastatic vs Control")
dev.off()
